# A Case Study: The Effect of Gun Ownership on Gun-Homicide Rates


We consider the problem of estimating the effect of gun ownership on the homicide rate. For this purpose, we perform inference on $\beta$ in the following the partially linear model:
$$
Y_{j, t}=\beta D_{j,(t-1)}+g\left(X_{j, t}, \bar{X}_j, \bar{X}_t, X_{j, 0}, Y_{j, 0}, t\right)+\epsilon_{j, t}
$$
$Y_{j, t}$ is the log homicide rate in county $j$ at time $t. D_{j, t-1}$ is the log fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership $G_{j, t}$, which is not observed. $X_{j, t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. We use $\bar{X}_j$ to denote the within county average of $X_{j, t}$ and $\bar{X}_t$ to denote the within time period average of $X_{j, t} . X_{j, 0}$ and $Y_{j, 0}$ denote initial conditions in county $j$. We use $Z_{j, t}$ to denote the set of observed control variables $\left\{X_{j, t}, \bar{X}_j, \bar{X}_t, X_{j, 0}, Y_{j, 0}, t\right\}$, so that our model is

$$
 Y_{i,t} = \beta D_{i,(t-1)} + g(Z_{i,t}) + \epsilon_{i,t}.
$$

## Data

$Y_{j,t}$ is the log homicide rate in county $j$ at time $t$, $D_{j, t-1}$ is the log fraction of suicides committed with a firearm in county $j$ at time $t-1$, which we use as a proxy for gun ownership,  and  $Z_{j,t}$ is a set of demographic and economic characteristics of county $j$ at time $t$. Assuming the firearm suicide rate is a good proxy for gun ownership, the parameter $\beta$ is the effect of gun ownership on homicide rates, controlling for county-level demographic and economic characteristics.

The sample covers 195 large United States counties between the years 1980 through 1999, giving us 3900 observations.

In [1]:
# Import relevant packages
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
warnings.simplefilter('ignore')

np.random.seed(1234)

In [3]:
file = "/content/sample_data/main.dta"
data = pd.read_stata(file)
data.shape

(1937, 73)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1937 entries, 0 to 1936
Data columns (total 73 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   province           1937 non-null   object 
 1   lati               1937 non-null   float32
 2   near_dist          1937 non-null   float32
 3   pat_per15          1937 non-null   float32
 4   gdpp               1937 non-null   float32
 5   samplenorth        1937 non-null   float32
 6   ele                1937 non-null   float32
 7   maizev             1937 non-null   float32
 8   ricewheat          1937 non-null   float32
 9   interricevm        1937 non-null   float32
 10  interwheatvm       1937 non-null   float32
 11  d                  1937 non-null   int8   
 12  edu                1937 non-null   float32
 13  above65            1937 non-null   float64
 14  migrenkou          1937 non-null   float32
 15  lower15            1937 non-null   float64
 16  r_pca1             1569 

In [7]:
data['interricevm'].describe()

,interricevm
count,1937.000000
mean,0.041127
std,0.053162
min,0.000000
25%,0.000567
50%,0.020750
75%,0.059994
max,0.204782


# Estimation

## Baseline OLS Estimates

After preprocessing the data, as a baseline model, we first look at simple regression of $Y_{j,t}$ on $D_{j,t-1}$ without controls in the full data set.

In [ ]:
# Baseline OLS
X = sm.add_constant(data['logfssl'])
y = usedata['pat_per15']
lm0 = sm.OLS(y, X).fit(cov_type='HC3')
vc0 = lm0.cov_params()
coef = lm0.params['logfssl']
std_err = np.sqrt(vc0.loc['logfssl', 'logfssl'])
print("Baseline OLS:", coef, "(", std_err, ")")

The point estimate is $0.302$ with the confidence interval ranging from 0.277 to 0.327. This
suggests that increases in gun ownership rates are related to gun homicide rates - if gun ownership increases by 1% then the predicted gun homicide rate goes up by 0.3%, without controlling for counties' characteristics.



Next we estimate with the baseline set of controls.

In [ ]:
# Regression on baseline controls
varlist = [d] + X1 + X2 + census
X = sm.add_constant(usedata[varlist])
y = usedata['logghomr']
lmC = sm.OLS(y, X).fit(cov_type='HC3')
vcC = lmC.cov_params()
coef = lmC.params['logfssl']
std_err = np.sqrt(vcC.loc['logfssl', 'logfssl'])
print("OLS with Controls:", coef, "(", std_err, ")")

<!-- Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics, we next include time and space averages. -->

We can also run our regression with time and space averages as controls.

In [ ]:
# Regression on time and cross-sectional averages
varlistX = X1 + X2 + census
varlistMeans = [d] + X1 + X2 + census

# Adding county and time means
for var in varlistX:
    varlistMeans.append(var + 'J')
    varlistMeans.append(var + 'T')

X = sm.add_constant(usedata[varlistMeans])
y = usedata['logghomr']
lmM = sm.OLS(y, X).fit(cov_type='HC3')
vcM = lmM.cov_params()
coef = lmM.params['logfssl']
std_err = np.sqrt(vcM.loc['logfssl', 'logfssl'])
print("OLS with Averages:", coef, "(", std_err, ")")

Since our goal is to estimate the effect of gun ownership after controlling for a rich set county characteristics, we now include all controls.

In [ ]:
# Regression on all controls

X = sm.add_constant(usedata.drop(columns=['logghomr']))
y = usedata['logghomr']
lmA = sm.OLS(y, X).fit(cov_type='HC3')
vcA = lmA.cov_params()
coef = lmA.params['logfssl']
std_err = np.sqrt(vcA.loc['logfssl', 'logfssl'])
print("OLS All:", coef, "(", std_err, ")")

After controlling for a rich set of characteristics, the point estimate of gun ownership attenuates to 0.179.

***NB***: In the background, `statsmodels` is dropping variables based on collinearity diagnostics. These depend on system linear algebra routines and can lead to large differences in high-dimensional or other ill-conditioned settings when using otherwise identical code across languages and/or machines.

Now we turn to our double machine learning framework, employing linear and flexible estimation methods with cross-fitting.

## DML Estimates

We perform inference on $\beta$ in the following the partially linear model:
 $$
Y_{j, t}=\beta D_{j,(t-1)}+g(Z_{j,t})+\epsilon_{j, t}.
$$
In the first stage, using cross-fitting, we employ modern regression methods to build estimators $\hat \ell(Z_{j,t})$ and $\hat m(Z_{j,t})$, where
- $\ell(Z_{j,t}):=E(Y_{j,t}|Z_{j,t})$
- $m(Z_{j,t}):=E(D_{j,t}|Z_{j,t})$

Using these, we obtain the estimates of the residualized quantities
- $\tilde Y_{j,t} = Y_{j,t}- E(Y_{j,t}|Z_{j,t})$
- $\tilde D_{j,t}= D_{j,t}- E(D_{j,t}|Z_{j,t})$

Using these residualized quantities, we note our model can be written as
$$
\tilde Y_{j,t} = \beta \tilde D_{j,t} + \epsilon_{j,t}, \quad E (\epsilon_{j,t} |\tilde D_{j,t}) =0.
$$
In the final stage, using ordinary least squares of $\tilde Y_{j,t}$ on $\tilde D_{j,t}$, we obtain the
estimate of $\beta$.

In the following, we consider 10 different methods for the first-stage models for $\ell(\cdot)$ and $m(\cdot)$ covering linear, penalized linear, and flexible methods. We also report the first-stage RMSE scores for estimating $Y$ and $D$.

In [ ]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    time: array of time indices, eg [0,1,...,T-1,0,1,...,T-1,...,0,1,...,T-1]
    clu: array of cluster indices, eg [1073, 1073, 1073, ..., 5055, 5055, 5055, 5055]
    cluster: bool, whether to use clustered standard errors

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)  # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1)  # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    dml_data = pd.concat([pd.Series(resy, name='resy'), pd.Series(resD, name='resD')], axis=1)
    ols_mod = smf.ols(formula='resy ~ 1 + resD', data=dml_data).fit()

    point = ols_mod.params[1]
    stderr = ols_mod.bse[1]
    epsilon = ols_mod.resid

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [ ]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point,  # point estimate
                         'stderr': stderr,  # standard error
                         'lower': point - 1.96 * stderr,  # lower end of 95% confidence interval
                         'upper': point + 1.96 * stderr,  # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)),  # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2))  # RMSE of model that predicts treatment D
                         }, index=[name])

In [ ]:
# OLS No Controls
Y = usedata['logghomr']
D = usedata['logfssl']
Z = pd.DataFrame({"Const": np.ones(len(Y))})  # regression on constant

modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_OLS = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = summary(*result_OLS, Z, D, y, name='OLS No Controls')
table

In [ ]:
# Basic Controls
basic_controls = [d] + X1 + X2 + census
Z = usedata[basic_controls].drop(columns=['logfssl'])

modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_basic = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_basic, Z, D, y, name='OLS Basic Controls')])
table

In [ ]:
# All Controls
# Regression on time and cross-sectional averages
varlistX = X1 + X2 + census
varlistMeans = [d] + X1 + X2 + census

# Adding county and time means
for var in varlistX:
    varlistMeans.append(var + 'J')
    varlistMeans.append(var + 'T')

Z = sm.add_constant(usedata[varlistMeans].drop(columns=['logfssl']))

modely = make_pipeline(StandardScaler(), LinearRegression())
modeld = make_pipeline(StandardScaler(), LinearRegression())

# Run DML model with nfolds folds of cross-fitting
result_all = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_all, Z, D, y, name='OLS All Controls')])
table

In [ ]:
# Now lets do Cross-validated Lasso, Ridge, ENet
cv = KFold(n_splits=10, shuffle=True, random_state=123)  # shuffled k-folds

In [ ]:
# Define LassoCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), LassoCV(cv=cv))
modeld = make_pipeline(StandardScaler(), LassoCV(cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_LassoCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_LassoCV, Z, D, y, name='LassoCV')])
table

In [ ]:
# Define RidgeCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), RidgeCV(cv=cv))
modeld = make_pipeline(StandardScaler(), RidgeCV(cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_RidgeCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_RidgeCV, Z, D, y, name='RidgeCV')])
table

In [ ]:
# Define ElasticNetCV models with n_splits folds of cross-validation
modely = make_pipeline(StandardScaler(), ElasticNetCV(l1_ratio=0.5, cv=cv))
modeld = make_pipeline(StandardScaler(), ElasticNetCV(l1_ratio=0.5, cv=cv))

# Run DML model with nfolds folds of cross-fitting
result_ENetCV = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_ENetCV, Z, D, y, name='ENetCV')])
table

In [ ]:
# DML with Random Forests. RFs don't require scaling but we do it for consistency
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_RF = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_RF, Z, D, y, name='RF')])
table

In [ ]:
# DML with Boosted Trees
modely = make_pipeline(StandardScaler(), GradientBoostingRegressor(max_depth=4, n_iter_no_change=5))
modeld = make_pipeline(StandardScaler(), GradientBoostingRegressor(max_depth=4, n_iter_no_change=5))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_BT = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_BT, Z, D, y, name='Boosted Trees')])
table

In [ ]:
# DML with NNs
modely = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50, 50, 50),
                                    activation='relu',
                                    solver='adam',
                                    alpha=0.0001,
                                    batch_size=200,
                                    learning_rate='constant',
                                    learning_rate_init=0.001,
                                    max_iter=200,
                                    shuffle=True,
                                    random_state=None,
                                    tol=1e-4,
                                    verbose=False,
                                    warm_start=False,
                                    momentum=0.9,
                                    nesterovs_momentum=True,
                                    early_stopping=True,
                                    validation_fraction=0.2,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    epsilon=1e-08,
                                    n_iter_no_change=10)
                       )
modeld = make_pipeline(StandardScaler(),
                       MLPRegressor(hidden_layer_sizes=(50, 50, 50, 50),
                                    activation='relu',
                                    solver='adam',
                                    alpha=0.0001,
                                    batch_size=200,
                                    learning_rate='constant',
                                    learning_rate_init=0.001,
                                    max_iter=200,
                                    shuffle=True,
                                    random_state=None,
                                    tol=1e-4,
                                    verbose=False,
                                    warm_start=False,
                                    momentum=0.9,
                                    nesterovs_momentum=True,
                                    early_stopping=True,
                                    validation_fraction=0.2,
                                    beta_1=0.9,
                                    beta_2=0.999,
                                    epsilon=1e-08,
                                    n_iter_no_change=10)
                       )

# Run DML model with nfolds folds of cross-fitting
result_NN = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_NN, Z, D, y, name='NN (Early Stopping)')])
table

In [ ]:
rmses = table.iloc[:, -2:]
rmses

In [ ]:
print("Lowest RMSE y: ", rmses.iloc[:, 0].idxmin())
print("Lowest RMSE D: ", rmses.iloc[:, 1].idxmin())

In [ ]:
# DML with best model, which is RF
modely = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))
modeld = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=100, min_samples_leaf=5, random_state=123))

# Run DML model with nfolds folds of cross-fitting (computationally intensive)
result_best = dml(Z, D, Y, modely, modeld, nfolds=5, classifier=False)
table = pd.concat([table, summary(*result_best, Z, D, y, name='Best')])
table

In [ ]:
# Least Squares Model Average
yhat_all = pd.concat([
    pd.Series(result_OLS[2]),
    pd.Series(result_basic[2]),
    pd.Series(result_all[2]),
    pd.Series(result_LassoCV[2]),
    pd.Series(result_RidgeCV[2]),
    pd.Series(result_ENetCV[2]),
    pd.Series(result_RF[2]),
    pd.Series(result_BT[2]),
    pd.Series(result_NN[2])
], axis=1)

Dhat_all = pd.concat([
    pd.Series(result_OLS[3]),
    pd.Series(result_basic[3]),
    pd.Series(result_all[3]),
    pd.Series(result_LassoCV[3]),
    pd.Series(result_RidgeCV[3]),
    pd.Series(result_ENetCV[3]),
    pd.Series(result_RF[3]),
    pd.Series(result_BT[3]),
    pd.Series(result_NN[3])
], axis=1)

ma_y = sm.OLS(usedata['logghomr'], yhat_all).fit()
ma_d = sm.OLS(usedata['logfssl'], Dhat_all).fit()

weights_y = ma_y.params
weights_d = ma_d.params

lm_k = sm.OLS(ma_y.resid, ma_d.resid).fit(cov_type='HC3')
lsma = pd.Series({"estimate": lm_k.params[0], "stderr": lm_k.bse[0]},
                 name="Least Squares Model Average").to_frame().T

final_table = table.iloc[:, :2]
final_table = pd.concat([final_table, lsma], axis=0)
final_table